# Evaluation of Fine-tuned Models
### This notebook evaluates all fine-tuned models and their corresponding pre-trained models on synthetic and real receipts.
### The evaluation based on their default setting:
* Text Detection: precision, recall, hmean, fps
* Text Recognition: accuracy, fps

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Installation

In [2]:
!python -m pip install paddlepaddle-gpu==2.4.2.post117 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.3/557.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-api-core 2.16.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you h

In [3]:
!git clone https://github.com/PaddlePaddle/PaddleOCR

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 49751, done.
remote: Counting objects: 100% (976/976), done.
remote: Compressing objects: 100% (616/616), done.
remote: Total 49751 (delta 395), reused 852 (delta 328), pack-reused 48775
Receiving objects: 100% (49751/49751), 346.85 MiB | 51.22 MiB/s, done.
Resolving deltas: 100% (34784/34784), done.


In [4]:
%cd PaddleOCR
!pip3 install -r requirements.txt
%cd ..

/content/PaddleOCR
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.4 MB/s eta 0:00:00
/content


In [5]:
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/det_mv3_east_v2.0_train.tar
%cd pretrain_models
!tar -xf det_mv3_east_v2.0_train.tar && rm -rf det_mv3_east_v2.0_train.tar
%cd ..

--2024-07-11 08:34:35--  https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/det_mv3_east_v2.0_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.47.176, 2409:8c04:1001:1203:0:ff:b0bb:4f27
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.47.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33679360 (32M) [application/x-tar]
Saving to: ‘./pretrain_models/det_mv3_east_v2.0_train.tar’

det_mv3_east_v2.0_t 100%[===================>]  32.12M  5.94MB/s    in 14s     

2024-07-11 08:34:50 (2.27 MB/s) - ‘./pretrain_models/det_mv3_east_v2.0_train.tar’ saved [33679360/33679360]

/content/pretrain_models
/content


In [6]:
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/multilingual/german_mobile_v2.0_rec_train.tar
!tar -xf german_mobile_v2.0_rec_train.tar

--2024-07-11 08:34:50--  https://paddleocr.bj.bcebos.com/dygraph_v2.0/multilingual/german_mobile_v2.0_rec_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.47.176, 2409:8c04:1001:1203:0:ff:b0bb:4f27
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.47.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19496960 (19M) [application/x-tar]
Saving to: ‘german_mobile_v2.0_rec_train.tar’

german_mobile_v2.0_ 100%[===================>]  18.59M  3.85MB/s    in 12s     

2024-07-11 08:35:04 (1.53 MB/s) - ‘german_mobile_v2.0_rec_train.tar’ saved [19496960/19496960]



In [7]:
!pip install paddleocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.3/407.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=04159c58d1292e24bd8ce14ae086704ddb7af5296924a2a287b0068f23f5151d
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [8]:
!pip3 install lanms-nova

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.3/219.3 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lanms-nova: filename=lanms_nova-1.0.3-cp310-cp310-linux_x86_64.whl size=1268298 sha256=f5df477ad7bf160e02ad54293622f907d2c7b5532916cd7586570f17c97b581f
  Stored in directory: /root/.cache/pip/wheels/61/ff/81/814bf77943755a77768709bba2eb39a0b5005d54dd22072e6d
Successfully built lanms-nova


In [9]:
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/Multilingual_PP-OCRv3_det_distill_train.tar
%cd pretrain_models
!tar -xf Multilingual_PP-OCRv3_det_distill_train.tar && rm -rf Multilingual_PP-OCRv3_det_distill_train.tar
%cd ..

--2024-07-11 08:35:28--  https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/Multilingual_PP-OCRv3_det_distill_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 36.110.192.178, 2409:8c04:1001:1203:0:ff:b0bb:4f27
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|36.110.192.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139335680 (133M) [application/x-tar]
Saving to: ‘./pretrain_models/Multilingual_PP-OCRv3_det_distill_train.tar’

Multilingual_PP-OCR 100%[===================>] 132.88M  18.7MB/s    in 8.4s    

2024-07-11 08:35:38 (15.9 MB/s) - ‘./pretrain_models/Multilingual_PP-OCRv3_det_distill_train.tar’ saved [139335680/139335680]

/content/pretrain_models
/content


# Fine-tuned models vs. Pre-trined models

### Evaluation Text Detection

In [ ]:
import subprocess
import re
import pandas as pd

# Function to run the evaluation command and capture the output
def evaluate_model(model_path, config_file):
    command = f"python3 /content/PaddleOCR/tools/eval.py -c {config_file} -o Global.checkpoints={model_path}"
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return result.stdout

# Function to extract metrics from the output
def extract_metrics(output):
    precision_matches = re.findall(r'precision:\s*([\d.]+)', output)
    recall_matches = re.findall(r'recall:\s*([\d.]+)', output)
    hmean_matches = re.findall(r'hmean:\s*([\d.]+)', output)
    fps_matches = re.findall(r'fps:\s*([\d.]+)', output)

    # Use the last occurrence if there are multiple matches
    precision = float(precision_matches[-1]) if precision_matches else None
    recall = float(recall_matches[-1]) if recall_matches else None
    hmean = float(hmean_matches[-1]) if hmean_matches else None
    fps = float(fps_matches[-1]) if fps_matches else None

    return {
        "Precision": precision,
        "Recall": recall,
        "H-mean": hmean,
        "FPS": fps
    }

# Function to evaluate a set of models
def evaluate_models(model_info):
    results = []
    for model_name, model_details in model_info.items():
        model_path = model_details["path"]
        config_file = model_details["config"]
        output = evaluate_model(model_path, config_file)
        #print(output)
        metrics = extract_metrics(output)
        metrics["Model"] = model_name
        results.append(metrics)

    return results

### Evaluate Pre-trained on Synthetic data

In [ ]:
models = {
    "east_pretrained": {
        "path": "/content/pretrain_models/det_mv3_east_v2.0_train/best_accuracy.pdparams",
        "config": "/content/drive/MyDrive/det_mv3_east.yml"
    },
    "db_pretrained": {
        "path": "/content/pretrain_models/Multilingual_PP-OCRv3_det_distill_train/best_accuracy.pdparams",
        "config": "/content/drive/MyDrive/ch_PP-OCRv3_det_cml.yml"
    }
}

# Call the function
results_syn = evaluate_models(models)

# Convert results to DataFrame (optional)
df1 = pd.DataFrame(results_syn, columns=["Model", "Precision", "Recall", "H-mean", "FPS"])
print(df1.to_string(index=False))

          Model  Precision   Recall   H-mean       FPS
east_pretrained   0.299752 0.550735 0.388211 17.738597
  db_pretrained   0.506312 0.645556 0.567518  2.038031


### Evaluate Pre-trained on Real data

In [ ]:
models = {
    "east_pretrained": {
        "path": "/content/pretrain_models/det_mv3_east_v2.0_train/best_accuracy.pdparams",
        "config": "/content/drive/MyDrive/det_mv3_east.yml"
    },
    "db_pretrained": {
        "path": "/content/pretrain_models/Multilingual_PP-OCRv3_det_distill_train/best_accuracy.pdparams",
        "config": "/content/drive/MyDrive/ch_PP-OCRv3_det_cml.yml"
    }
}

# Call the function
results_syn = evaluate_models(models)

# Convert results to DataFrame (optional)
df1 = pd.DataFrame(results_syn, columns=["Model", "Precision", "Recall", "H-mean", "FPS"])
print(df1.to_string(index=False))

          Model  Precision   Recall   H-mean       FPS
east_pretrained   0.120672 0.253931 0.163600 24.968463
  db_pretrained   0.269965 0.426101 0.330521  2.319506


###  DB & EAST  trained on synthetic data, validate on synthetic data

In [ ]:
models = {
    "east_finetune": {
        "path": "/content/drive/MyDrive/training/east_300syn_syn/output/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/det_mv3_east.yml"
    },
    "db_finetuned": {
        "path": "/content/drive/MyDrive/training/db_300syn_syn/output/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/ch_PP-OCRv3_det_cml.yml"
    }
}

# Call the function
results_syn = evaluate_models(models)

# Convert results to DataFrame (optional)
df1 = pd.DataFrame(results_syn, columns=["Model", "Precision", "Recall", "H-mean", "FPS"])
print(df1.to_string(index=False))


Evaluation on synthetic receipt
        Model  Precision   Recall   H-mean       FPS
east_finetune   0.845291 0.923373 0.882609 18.133843
 db_finetuned   0.995191 0.941218 0.967452  2.017783


###  DB & EAST  trained on synthetic data, validate on synthetic data, evaluate on real data

In [ ]:
models = {
    "east_finetune": {
        "path": "/content/drive/MyDrive/training/east_300syn_syn/output/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/east_syn_syn.yml"
    },
    "db_finetuned": {
        "path": "/content/drive/MyDrive/training/db_300syn_syn/output/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/db_syn_syn.yml"
    }
}

# Call the function
results_syn = evaluate_models(models)

# Convert results to DataFrame (optional)
df1 = pd.DataFrame(results_syn, columns=["Model", "Precision", "Recall", "H-mean", "FPS"])
print(df1.to_string(index=False))

        Model  Precision  Recall   H-mean       FPS
east_finetune   0.395028 0.64544 0.490100 25.078851
 db_finetuned   0.871494 0.70021 0.776518  2.212190


#### further fine-tuned the models on real data

In [ ]:
models = {
    "east_optimized": {
        "path": "/content/drive/MyDrive/training/east_300syn_syn/output_finetune/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/east_syn_syn.yml"
    },
    "db_optimized": {
        "path": "/content/drive/MyDrive/training/db_300syn_syn/output_finetuned/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/db_syn_syn.yml"
    }
}

# Call the function
results_syn = evaluate_models(models)

# Convert results to DataFrame (optional)
df1 = pd.DataFrame(results_syn, columns=["Model", "Precision", "Recall", "H-mean", "FPS"])
print(df1.to_string(index=False))

         Model  Precision   Recall   H-mean       FPS
east_optimized   0.851879 0.902778 0.876590 24.662439
  db_optimized   0.962539 0.814727 0.882487  2.255888


###  DB & EAST  trained on synthetic data, validate on real data

In [ ]:
models = {
    "east_finetune": {
        "path": "/content/drive/MyDrive/training/east_300syn_real/output_lr0.001/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/east_syn_real.yml"
    },
    "db_finetuned": {
        "path": "/content/drive/MyDrive/training/db_300syn_real/output/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/db_syn_real.yml"
    }
}

# Call the function
results_syn = evaluate_models(models)

# Convert results to DataFrame (optional)
df1 = pd.DataFrame(results_syn, columns=["Model", "Precision", "Recall", "H-mean", "FPS"])
print(df1.to_string(index=False))

        Model  Precision   Recall   H-mean       FPS
east_finetune   0.449516 0.694182 0.545679 24.888366
 db_finetuned   0.906092 0.705451 0.793281  2.205200


###  DB & EAST  trained on real data, validate on real data

In [ ]:
models = {
    "east_finetune": {
        "path": "/content/drive/MyDrive/training/east_300real/output/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/east_real_real.yml"
    },
    "db_finetuned": {
        "path": "/content/drive/MyDrive/training/db_300real/output/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/ch_PP-OCRv3_det_cml.yml"
    }
}

# Call the function
results_syn = evaluate_models(models)

# Convert results to DataFrame (optional)
df1 = pd.DataFrame(results_syn, columns=["Model", "Precision", "Recall", "H-mean", "FPS"])
print(df1.to_string(index=False))

        Model  Precision   Recall   H-mean       FPS
east_finetune   0.898654 0.927149 0.912679 24.711438
 db_finetuned   0.973791 0.817872 0.889047  2.200959


###  DB & EAST  trained on mixed data, validate on real data


In [ ]:
models = {
    "east_finetune": {
        "path": "/content/drive/MyDrive/training/east_combi/output/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/east_combi.yml"
    },
    "db_finetuned": {
        "path": "/content/drive/MyDrive/training/db_combi/output/ch_PP-OCR_v3_det/best_model/model",
        "config": "/content/drive/MyDrive/db_combi.yml"
    }
}

# Call the function
results_syn = evaluate_models(models)

# Convert results to DataFrame (optional)
df1 = pd.DataFrame(results_syn, columns=["Model", "Precision", "Recall", "H-mean", "FPS"])
print(df1.to_string(index=False))

        Model  Precision   Recall   H-mean       FPS
east_finetune   0.915068 0.937369 0.926084 25.277332
 db_finetuned   0.950462 0.970388 0.960322  2.210454


### Evaluate CRNN pretrained model on synthetic data

In [ ]:
# Function to run the evaluation command and capture the output
def evaluate_model(model_path, config_file):
    command = f"python3 /content/PaddleOCR/tools/eval.py -c {config_file} -o Global.checkpoints={model_path}"
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return result.stdout

# Function to extract metrics from the CRNN
def extract_metrics2(output):
    acc_matches = re.findall(r'acc:\s*([\d.]+)', output)
    fps_matches = re.findall(r'fps:\s*([\d.]+)', output)

    # Use the last occurrence if there are multiple matches
    acc = float(acc_matches[-1]) if acc_matches else None
    fps = float(fps_matches[-1]) if fps_matches else None

    return {
        "Accuracy": acc,
        "FPS": fps
    }

# Function to evaluate a set of models
def evaluate_models(model_info):
    results = []
    for model_name, model_details in model_info.items():
        model_path = model_details["path"]
        config_file = model_details["config"]
        output = evaluate_model(model_path, config_file)
        #print(output)
        metrics = extract_metrics2(output)
        metrics["Model"] = model_name
        results.append(metrics)

    return results

In [ ]:
models = {
    "crnn_pretrained": {"path": "/content/best_accuracy", "config": "/content/drive/MyDrive/crnn_syn_syn.yml"},
}
results_df = evaluate_models(models)
print(results_df)

[{'Accuracy': 0.0, 'FPS': 1849.451056847065, 'Model': 'crnn_pretrained'}]


### Evaluate CRNN pre-trained model on real data

In [ ]:
models = {
    "crnn_pretrained": {"path": "/content/best_accuracy", "config": "/content/drive/MyDrive/crnn_syn_real.yml"},
}
results_df = evaluate_models(models)
print(results_df)

[{'Accuracy': 0.0, 'FPS': 2453.9660949249505, 'Model': 'crnn_pretrained'}]


### CRNN fine-tuned model on synthetic data, validate on synthetic data

In [ ]:
models = {
    "crnn_finetuned": {"path": "/content/drive/MyDrive/training/crnn_300syn_syn/output/ch_PP-OCR_v3_rec/best_model/model", "config": "/content/drive/MyDrive/crnn_syn_syn.yml"},
}
results_df = evaluate_models(models)
print(results_df)

[{'Accuracy': 0.9042857078265307, 'FPS': 1740.3591134758394, 'Model': 'crnn_finetuned'}]


###  CRNN fine-tuned on synthetic data, validate on synthetic data, evaluate on real data

In [ ]:
models = {
    "crnn_finetuned": {"path": "/content/drive/MyDrive/training/crnn_300syn_syn/output/ch_PP-OCR_v3_rec/best_model/model", "config": "/content/drive/MyDrive/crnn_syn_real.yml"},
}
results_df = evaluate_models(models)
print(results_df)

[{'Accuracy': 0.13007499319866822, 'FPS': 2399.179331824938, 'Model': 'crnn_finetuned'}]


### CRNN fine-tuned model on synthetic data, validate on real data

In [ ]:
models = {
    "crnn_finetuned": {"path": "/content/drive/MyDrive/training/crnn_300syn_real/output/ch_PP-OCR_v3_rec/best_model/model", "config": "/content/drive/MyDrive/crnn_syn_real.yml"},
}
results_df = evaluate_models(models)
print(results_df)

[{'Accuracy': 0.17119213816604048, 'FPS': 2404.0187449559235, 'Model': 'crnn_finetuned'}]


### CRNN fine-tuned model on real data, validate on real data

In [ ]:
models = {
    "crnn_finetuned": {"path": "/content/drive/MyDrive/training/crnn_300real/output/ch_PP-OCR_v3_rec/best_model/model", "config": "/content/drive/MyDrive/crnn_syn_real.yml"},
}
results_df = evaluate_models(models)
print(results_df)

[{'Accuracy': 0.8303594496241028, 'FPS': 2356.05311405408, 'Model': 'crnn_finetuned'}]


### Evaluate CRNN fine-tuned model on mixed data, validate on real data

In [ ]:
models = {
    "crnn_finetuned": {"path": "/content/drive/MyDrive/training/crnn_combi/output/ch_PP-OCR_v3_rec/best_model/model", "config": "/content/drive/MyDrive/rec_combi.yml"},
}
results_df = evaluate_models(models)
print(results_df)

[{'Accuracy': 0.8236358913275513, 'FPS': 2367.2681980554667, 'Model': 'crnn_finetuned'}]


# Test different combination of OCR pipeline

### regular receipt

#### EAST + CRNN

In [17]:
!python3 /content/PaddleOCR/tools/infer/predict_system.py --image_dir="/content/drive/MyDrive/testdata/img/IMG_1046.png" --det_model_dir="/content/drive/MyDrive/inference/east_combi" --rec_model_dir="/content/drive/MyDrive/inference/crnn_real/" --rec_image_shape="3, 32, 100" --rec_char_dict_path="/content/PaddleOCR/ppocr/utils/dict/german_dict.txt" --det_algorithm="EAST" --vis_font_path="/content/drive/MyDrive/Roboto-Bold.ttf" --use_gpu="false"

/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr

#### DB + CRNN

In [20]:
!python3 /content/PaddleOCR/tools/infer/predict_system.py --image_dir="/content/drive/MyDrive/testdata/img/IMG_1046.png" --det_model_dir="/content/drive/MyDrive/inference/db_combi/Student" --rec_model_dir="/content/drive/MyDrive/inference/crnn_real/" --rec_image_shape="3, 32, 100" --rec_char_dict_path="/content/PaddleOCR/ppocr/utils/dict/german_dict.txt" --vis_font_path="/content/drive/MyDrive/Roboto-Bold.ttf"

/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr

### fold receipt

#### EAST + CRNN

In [23]:
!python3 /content/PaddleOCR/tools/infer/predict_system.py --image_dir="/content/drive/MyDrive/testdata/img/IMG_1074.png" --det_model_dir="/content/drive/MyDrive/inference/east_combi" --rec_model_dir="/content/drive/MyDrive/inference/crnn_real/" --rec_image_shape="3, 32, 100" --rec_char_dict_path="/content/PaddleOCR/ppocr/utils/dict/german_dict.txt" --det_algorithm="EAST" --vis_font_path="/content/drive/MyDrive/Roboto-Bold.ttf" --use_gpu="false"

/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr

#### DB + CRNN

In [29]:
!python3 /content/PaddleOCR/tools/infer/predict_system.py --image_dir="/content/drive/MyDrive/testdata/img/IMG_1074.png" --det_model_dir="/content/drive/MyDrive/inference/db_combi/Student" --rec_model_dir="/content/drive/MyDrive/inference/crnn_real/" --rec_image_shape="3, 32, 100" --rec_char_dict_path="/content/PaddleOCR/ppocr/utils/dict/german_dict.txt" --vis_font_path="/content/drive/MyDrive/Roboto-Bold.ttf"

/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr

### blurr receipt

In [26]:
!python3 /content/PaddleOCR/tools/infer/predict_system.py --image_dir="/content/drive/MyDrive/testdata/img/IMG_1207.png" --det_model_dir="/content/drive/MyDrive/inference/east_combi" --rec_model_dir="/content/drive/MyDrive/inference/crnn_real/" --rec_image_shape="3, 32, 100" --rec_char_dict_path="/content/PaddleOCR/ppocr/utils/dict/german_dict.txt" --det_algorithm="EAST" --vis_font_path="/content/drive/MyDrive/Roboto-Bold.ttf" --use_gpu="false"

/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr

In [30]:
!python3 /content/PaddleOCR/tools/infer/predict_system.py --image_dir="/content/drive/MyDrive/testdata/img/IMG_1207.png" --det_model_dir="/content/drive/MyDrive/inference/db_combi/Student" --rec_model_dir="/content/drive/MyDrive/inference/crnn_real/" --rec_image_shape="3, 32, 100" --rec_char_dict_path="/content/PaddleOCR/ppocr/utils/dict/german_dict.txt" --vis_font_path="/content/drive/MyDrive/Roboto-Bold.ttf"

/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr